KMP (Knuth-Morris-Pratt)
===
<p style="text-align: right; font-size: 16px; padding-right: 350px;">d(*￣▽￣)=====b </p>
<h2 style="text-align: right; font-size: 16px;">D.E.Knuth，J.H.Morris and V.R.Pratt</h2>
___



暴力算法过程 Σ( ° △ °|||)︴

    if T[0] == W[0]
        do T[next] == W[next]
            until T[x] == W[end]
                success
            or T[x] != W[x]
                if T[1] == W[0] // 匹配失败时，文本串只从T[0]移动到T[1]
                    ....
                    

___

## 经验
    进行过匹配的字符串，对于我们而言就是“可知”的，它包含了一些有用的信息，这些信息将有助我们去规避暴力算法的低效性，即我们可以从这些失败的匹配经验中吸取教训，从而做的更好。
    
    那么，在匹配过的字符串中包含了什么信息呢 (＠_＠;)？
    对于给定的
        模式串 W:  chinchia
        文本串 T:  chianchinchichinchia
                               ^^^^^^^^
                               
___
    进过几次失败匹配后，我们来到了如下位置
        T:  chianchinchichinchia
        W:       chinchi...
        
        
___
    可想而知，下一个字符将匹配失败
    但从中，我们可以发现，在匹配过的字符串chinchi中，前后都有相同的字符 chi, 暂且将其称为字符串（已匹配过）的前缀和后缀。
        假使我们可以做如下移动
            T:  chianchinchichinchia
            W:       chinchi...
         -> W:           chi...
         即将前缀位置移动到后缀的位置，这是由于前缀与后缀具有相同的形式（chi*），后缀与文本串相匹配，前缀自然也会与其相匹配，之后，再由 * 重新开始继续匹配字符串。
         从而我们跳过了其中几个字符的匹配过程，这也就突破了爆力算法每次只能前进一步的缺陷。
         
___

至此，我们稍做总结 ~(￣0￣)/

    在已匹配过的字符串中，可能存在两个相同的子字符串--前缀和后缀，它可以是chinc中的c，也可以是chinch中的ch。我们可以在一次失败匹配后，将前缀的位置移动到后缀的位置处。
    前缀的位置是从 首字符开始，具体的长度则是由后缀所决定的；后缀可以在首字符后任意位置，长度受已匹配字符数影响。
        例如
            如果已匹配的字符串是 chinc, 则前后缀是 c， 如果是 chinch的话，则是 ch, 
            可以在失败匹配后直接将模式串移动到后缀位置，从前缀的ch之后继续匹配

            而 chinc 中的 hin 因为没有与其对应的前缀，所以它们在遇到失败匹配后，只能从头开始（c）匹配，它的意义在于为我们积累“经验”。
         

## 实现
    由上，我们知道了前缀可移动到遇到失败匹配的后缀处再继续进行匹配，从而减少不必要的匹配过程这一事实。 *´∀`)´∀`)*´∀`)*´∀`)
    
___
那么，如何通过代码将我们知道的信息，实现为具体的高效的算法呢？

    实时上非常简单
        c h i n c h i a
       -1 0 0 0 0 1 2 0
    只要我们找到每个字符所对应的前缀index,将其记录在案，那么我们就可以通过记录，查找失败匹配的前缀（或是从头开始）进行匹配。
    
___
```
/*
    我们将这种记录称为 next表， 由此，构造函数生成模式串的next表
*/
#include<stdio.h>
#include<string.h>
#include<stdlib.h>

int * getNext(char *s){
    int str_len = strlen(s);
    int *next = malloc( sizeof(int) * str_len );
    int i = 0;

    // t 可以简单表示为前/后缀的index   
    int t = next[0] = -1;

    // -1, 首项已给出
    while( i < str_len -1 ){ 
        if(t < 0 || s[t] == s[i] ){
            next[ ++i ] = ++t; // 如匹配 c 后，继续匹配 h
        }else{ // 失配
            t = next[t];
        }
    }
    return next;
}

int main(void){
    char *s = "chinchia";
    int *n = getNext(s);
    int i = 0;
    for( ; i < strlen(s); i++){
        printf("%d ", n[i]);
    }
    return 0;
}
```

___

## 完整程序

___

```
#include<stdio.h>
#include<string.h>
#include<stdlib.h>

int * getNext(char *s){
    int str_len = strlen(s);
    int *next = malloc( sizeof(int) * str_len );
    int i = 0;

    int t = next[0] = -1;
    while( i < str_len - 1 ){ 
        if(t < 0 || s[t] == s[i] ){
            i++;
            t++;
            next[i] = s[t] != s[i] ? t: next[t];
        }else{
            t = next[t];
        }
    }
    return next;
}


int match(char*s, char *t){
    int *next = getNext(t);
    int len_s = strlen(s), s_i = 0;
    int len_t = strlen(t), t_i = 0; // 模式串 

    while( t_i < len_t && s_i < len_s ){
        if( 0>t_i || t[t_i] == s[s_i] ){
            s_i++;
            t_i++;
        }else{
            t_i = next[t_i];
        }
    }
    free(next);
    return s_i - t_i;
}

int main(void){
    char *t = "chinchia";
    char *s = "chianchinchichinchia";
    printf("%d", match(s, t));
    return 0;
}
```